# Web Scrapping of Groww Website for Mutual Fund Details

AIM-

1.To Get Mutual Fund Details (MF Name, NAV,MIN SIP, MF rank,Fund Size,Fund Manager,Exit Load,Tax implication,Expense ratio etc.)

2.To Get the Details of Asset Allocation in Various Sector with percentage of Allocation.

# Step 1- Use the requests library to download web pages

In [1]:
import requests
import bs4
import pandas as pd

In [2]:
response=requests.get("https://groww.in/mutual-funds/quant-small-cap-fund-direct-plan-growth")
response

<Response [200]>

In [3]:
type(response)

requests.models.Response

In [4]:
page_content=response.text
page_content[:100]

'<!DOCTYPE html><html lang="en" data-theme="light"><head><meta charSet="utf-8" /><meta name="viewport'

# Use Beautiful Soup to parse and extract information

In [5]:
from bs4 import BeautifulSoup

In [6]:
html=BeautifulSoup(page_content,"html.parser")

In [7]:
type(html)

bs4.BeautifulSoup

# Getting Mutual Fund key Details Details

# Mutual Fund Name

In [8]:
MF_Name_selector="mfh239SchemeName fw400 fs24"
MF_Name=html.find_all("h1",class_=MF_Name_selector)

In [9]:
MF_Name[0].text

'Quant Small Cap Fund Direct Plan Growth'

# Mutual Fund NAV Date, NAV ,Min. SIP, Fund Size

In [10]:
MF_NAV_date_selection="fd12Header clrSubText fs16 fw400"
MF_NAV_date=html.find_all("td",class_=MF_NAV_date_selection)

In [11]:
MF_NAV_date[0].text

' NAV: 06 Apr 2023'

In [12]:
MF_NAV_selection="fd12Cell clrText130 fs16 fw500"
MF_NAV=html.find_all("td",class_=MF_NAV_selection)

In [13]:
MF_NAV[0].text

'₹150.20'

In [14]:
MF_Min_SIP_selection="fd12Cell clrText130 fs16 fw500"
MF_Min_SIP=html.find_all("td",class_=MF_NAV_selection)

In [15]:
MF_NAV[1].text

'₹1,000'

In [16]:
MF_fund_size_selection="fd12Cell clrText130 fs16 fw500"
MF_fund_size=html.find_all("td",class_=MF_NAV_selection)

In [17]:
MF_fund_size[2].text

'₹3,301.73Cr'

In [94]:
MF_Insignt_4={
    "Mutual Fund Name":MF_Name[0].text,
    "Mutual NAV Date":MF_NAV[0].text,
    "Mutual NAV ":MF_NAV[1].text,
    "Mutual Fund Size":MF_fund_size[2].text
    
    
}
MF_Insignt_4

{'Mutual Fund Name': 'Quant Small Cap Fund Direct Plan Growth',
 'Mutual NAV Date': '₹150.20',
 'Mutual NAV ': '₹1,000',
 'Mutual Fund Size': '₹3,301.73Cr'}

# Mutual Fund Asset Allocation Link 

In [19]:
Base_link="https://groww.in"
MF_fund_holding_link_selector="pc543Links clrText cur-po"
MF_fund_holding_link=html.find_all("a",class_=MF_fund_holding_link_selector)

In [20]:
MF_fund_holding_link[2]["href"]

'/stocks/rbl-bank-ltd'

In [21]:
MF_link_list=[]
for i in range(len(MF_fund_holding_link)):
    MF_fund_holding_link[0]["href"]
    MF_link_list.append(Base_link+(MF_fund_holding_link[i]["href"]))
MF_link_list

['https://groww.in/stocks/itc-ltd',
 'https://groww.in/stocks/jindal-stainless-ltd',
 'https://groww.in/stocks/rbl-bank-ltd',
 'https://groww.in/stocks/irb-infrastructure-developers-ltd',
 'https://groww.in/stocks/reliance-industries-ltd',
 'https://groww.in/stocks/punjab-national-bank',
 'https://groww.in/stocks/bikaji-foods-international-ltd',
 'https://groww.in/stocks/himachal-futuristic-communications-ltd',
 'https://groww.in/stocks/hindustan-copper-ltd',
 'https://groww.in/stocks/archean-chemical-industries-ltd']


# Mutual Fund Asset Allocation Details (MF Asset,Sector,Instrument,Asset Allocation)

In [22]:
MF_fund_sector_selection="fs14"
MF_fund_sector=html.find_all("td",class_=MF_fund_sector_selection)

In [23]:
MF_fund_sector_selection="fs14"
MF_fund_sector=html.find_all("td",class_=MF_fund_sector_selection)

MF_holding_details={
    "MF Asset":[],
    "Sector":[],
    "Instrument":[],
    "Asset Allocation":[]
}



for i in range(len(MF_fund_sector)):
    if i%4==0:
        MF_holding_details["MF Asset"].append((MF_fund_sector[i].text))
    elif i%4==1:
        MF_holding_details["Sector"].append((MF_fund_sector[i].text))
    elif i%4==2:
        MF_holding_details["Instrument"].append((MF_fund_sector[i].text))
    elif i%4==3:
        MF_holding_details["Asset Allocation"].append((MF_fund_sector[i].text))

MF_holding_details
    

{'MF Asset': ['ITC Ltd.',
  'Jindal Stainless Ltd.',
  'RBL Bank Ltd.',
  'IRB Infrastructure Developers Ltd.',
  'Reliance Industries Ltd.',
  'Punjab National Bank',
  'Bikaji Foods International Ltd.',
  'HFCL Ltd.',
  'Hindustan Copper Ltd.',
  'Archean Chemical Industries Ltd.'],
 'Sector': ['Consumer Staples',
  'Metals & Mining',
  'Financial',
  'Construction',
  'Energy',
  'Financial',
  'Consumer Staples',
  'Communication',
  'Metals & Mining',
  'Chemicals'],
 'Instrument': ['Equity',
  'Equity',
  'Equity',
  'Equity',
  'Equity',
  'Equity',
  'Equity',
  'Equity',
  'Equity',
  'Equity'],
 'Asset Allocation': ['9.5%',
  '5.7%',
  '5.3%',
  '5.0%',
  '4.4%',
  '4.4%',
  '4.1%',
  '2.5%',
  '2.4%',
  '2.2%']}

In [24]:
Holding_Info=pd.DataFrame(MF_holding_details)
Holding_Info

,MF Asset,Sector,Instrument,Asset Allocation
0,ITC Ltd.,Consumer Staples,Equity,9.5%
1,Jindal Stainless Ltd.,Metals & Mining,Equity,5.7%
2,RBL Bank Ltd.,Financial,Equity,5.3%
3,IRB Infrastructure Developers Ltd.,Construction,Equity,5.0%
4,Reliance Industries Ltd.,Energy,Equity,4.4%
5,Punjab National Bank,Financial,Equity,4.4%
6,Bikaji Foods International Ltd.,Consumer Staples,Equity,4.1%
7,HFCL Ltd.,Communication,Equity,2.5%
8,Hindustan Copper Ltd.,Metals & Mining,Equity,2.4%
9,Archean Chemical Industries Ltd.,Chemicals,Equity,2.2%


# Mutual Fund Details Such As Min. Investment, Min. SIP, MF Rank, Asset under Mgt,Date of incorpation,Fund Bench mark

In [25]:
MF_fund_equity_selection="fs16 fw500"
MF_fund_equity_allocation=html.find_all("td",class_=MF_fund_equity_selection)

In [26]:
MF_fund_equity_allocation[0].text

'₹5,000'

In [27]:
MF_Insight_1={
    "Min Investment":MF_fund_equity_allocation[0].text,
    "Min 2nd Investment":MF_fund_equity_allocation[1].text,
    "Min SIP":MF_fund_equity_allocation[2].text,
    "MF Rank (In Total Asset)":MF_fund_equity_allocation[3].text,
    "Asset under Mgt":MF_fund_equity_allocation[4].text,
    "Date of Incorporation":MF_fund_equity_allocation[5].text,
    "Fund bench mark":MF_fund_equity_allocation[6].text
    
}






MF_Insight_1

{'Min Investment': '₹5,000',
 'Min 2nd Investment': '₹1,000',
 'Min SIP': '₹1,000',
 'MF Rank (In Total Asset)': '#34 in India',
 'Asset under Mgt': '₹6,178.85Cr',
 'Date of Incorporation': '15 Apr 1996',
 'Fund bench mark': 'NIFTY Smallcap 250 Total Return Index'}

In [28]:
insight_2_selector="fs16"
insight_2=html.find_all("p",class_=insight_2_selector)

# Mutual Fund Exit load , Stamped Duty & Tax Implication Details

In [29]:
MF_Insight_2={
    "Exit load":insight_2[1].text,
    "Stamped Duty":insight_2[2].text,
    "Tax Implication":insight_2[3].text
    
}
MF_Insight_2

{'Exit load': 'Exit load of 1% if redeemed within 1 year',
 'Stamped Duty': '0.005% (from July 1st, 2020)',
 'Tax Implication': 'Returns are taxed at 15%, if you redeem before one year. After 1 year, you are required to pay LTCG tax of 10% on returns of Rs 1 lakh+ in a financial year. '}

In [30]:
expense_ratio_selector="fs16 fw500 ot654subHeading"
MF_fund_Expense_ratio=html.find_all("h3",class_=expense_ratio_selector)

In [31]:
MF_fund_Expense_ratio[0].text.strip("Expense ratio: ")

'0.62%'

In [32]:
expense_ratio=MF_fund_Expense_ratio[0].text.strip("Expense ratio: ")
expense_ratio

'0.62%'

In [33]:
fund_mgr_Sel="fm982PersonName fs16 fw500 clrText130"
MF_MGR=html.find_all("div",class_=fund_mgr_Sel)

In [34]:
MF_MGR[0].text

'Sanjeev Sharma'

In [35]:
MF_Insight_3={
    "Expense Ratio":MF_fund_Expense_ratio[0].text.strip("Expense ratio: "),
    "Mutual Fund Manager":MF_MGR[0].text
}
MF_Insight_3

{'Expense Ratio': '0.62%', 'Mutual Fund Manager': 'Sanjeev Sharma'}

# Mutual fund Address Details

In [36]:
MF_Add_Details_selector="fs16 clrText"
MF_Add_Details=html.find_all("div",class_=MF_Add_Details_selector)
MF_Add_Details[0].text

'6th Floor, Sea Breeze Building,Appa Saheb Marathe Marg,Prabhadevi, Mumbai 400025'

In [37]:
fund_house_details={
    "Address":MF_Add_Details[0].text,
    "Phone":MF_Add_Details[1].text,
    "Launch Date":MF_Add_Details[2].text,
    "E-Mail":MF_Add_Details[3].text
}

fund_house_details

{'Address': '6th Floor, Sea Breeze Building,Appa Saheb Marathe Marg,Prabhadevi, Mumbai 400025',
 'Phone': '022-62955000',
 'Launch Date': '15 Apr 1996',
 'E-Mail': 'NA'}

# Mutual Fund All Details 

In [38]:
Mutual_fund_Details={
    "Mutual Fund Name":MF_Name[0].text,
    "Mutual NAV Date":MF_NAV[0].text,
    "Mutual NAV Date":MF_NAV_date[0].text,
    "Mutual Fund Size":MF_fund_size[2].text,
    "Min Investment":MF_fund_equity_allocation[0].text,
    "Min 2nd Investment":MF_fund_equity_allocation[1].text,
    "Min SIP":MF_fund_equity_allocation[2].text,
    "MF Rank (In Total Asset)":MF_fund_equity_allocation[3].text,
    "Asset under Mgt":MF_fund_equity_allocation[4].text,
    "Date of Incorporation":MF_fund_equity_allocation[5].text,
    "Fund bench mark":MF_fund_equity_allocation[6].text,
    "Exit load":insight_2[1].text,
    "Expense Ratio":MF_fund_Expense_ratio[0].text.strip("Expense ratio: "),
    "Launch Date":MF_Add_Details[2].text,
    "E-Mail":MF_Add_Details[3].text,
    "Mutual Fund Manager":MF_MGR[0].text,
    "Stamped Duty":insight_2[2].text,
    "Tax Implication":insight_2[3].text,
    "Address":MF_Add_Details[0].text,
    "Phone":MF_Add_Details[1].text,
    
    
    
}
Mutual_fund_Details

{'Mutual Fund Name': 'Quant Small Cap Fund Direct Plan Growth',
 'Mutual NAV Date': ' NAV: 06 Apr 2023',
 'Mutual Fund Size': '₹3,301.73Cr',
 'Min Investment': '₹5,000',
 'Min 2nd Investment': '₹1,000',
 'Min SIP': '₹1,000',
 'MF Rank (In Total Asset)': '#34 in India',
 'Asset under Mgt': '₹6,178.85Cr',
 'Date of Incorporation': '15 Apr 1996',
 'Fund bench mark': 'NIFTY Smallcap 250 Total Return Index',
 'Exit load': 'Exit load of 1% if redeemed within 1 year',
 'Expense Ratio': '0.62%',
 'Launch Date': '15 Apr 1996',
 'E-Mail': 'NA',
 'Mutual Fund Manager': 'Sanjeev Sharma',
 'Stamped Duty': '0.005% (from July 1st, 2020)',
 'Tax Implication': 'Returns are taxed at 15%, if you redeem before one year. After 1 year, you are required to pay LTCG tax of 10% on returns of Rs 1 lakh+ in a financial year. ',
 'Address': '6th Floor, Sea Breeze Building,Appa Saheb Marathe Marg,Prabhadevi, Mumbai 400025',
 'Phone': '022-62955000'}

In [39]:
df=pd.DataFrame([Mutual_fund_Details]).T
df

,0
Mutual Fund Name,Quant Small Cap Fund Direct Plan Growth
Mutual NAV Date,NAV: 06 Apr 2023
Mutual Fund Size,"₹3,301.73Cr"
Min Investment,"₹5,000"
Min 2nd Investment,"₹1,000"
Min SIP,"₹1,000"
MF Rank (In Total Asset),#34 in India
Asset under Mgt,"₹6,178.85Cr"
Date of Incorporation,15 Apr 1996
Fund bench mark,NIFTY Smallcap 250 Total Return Index


# Final Code for getting the Mutual Fund All Details 

In [92]:
    import os
    import bs4
    from bs4 import BeautifulSoup
    import requests
    import pandas as pd

    topic_url=input("Enter the URL ")

    def get_topic_page(topic_url):
        # Download the page
        response = requests.get(topic_url)
        page_content = response.text
        # Check successful response
        if response.status_code != 200:
            raise Exception('Failed to load page {}'.format(topic_url))
        # Parse using Beautiful soup
        html= BeautifulSoup(page_content, 'html.parser')
        
        # Getting Mutual Fund Name 
        MF_Name_selector="mfh239SchemeName fw400 fs24"
        MF_Name=html.find_all("h1",class_=MF_Name_selector)
        
        
        # Getting Mutual Nav Details
        MF_NAV_date_selection="fd12Header clrSubText fs16 fw400"
        MF_NAV_date=html.find_all("td",class_=MF_NAV_date_selection)
        

        # Getting Mutual Fund Min SIP
        MF_Min_SIP_selection="fd12Cell clrText130 fs16 fw500"
        MF_Min_SIP=html.find_all("td",class_=MF_Min_SIP_selection)
        

        # Getting Mutual Fund Size
        MF_fund_size_selection="fd12Cell clrText130 fs16 fw500"
        MF_fund_size=html.find_all("td",class_=MF_NAV_selection)
        
        
        # Getting Mutual Fund Equity Allocation
        MF_fund_equity_selection="fs16 fw500"
        MF_fund_equity_allocation=html.find_all("td",class_=MF_fund_equity_selection)


        insight_2_selector="fs16"
        insight_2=html.find_all("p",class_=insight_2_selector)

        # Getting Mutual Fund Expense ratio
        expense_ratio_selector="fs16 fw500 ot654subHeading"
        MF_fund_Expense_ratio=html.find_all("h3",class_=expense_ratio_selector)
        expense_ratio=MF_fund_Expense_ratio[0].text.strip("Expense ratio: ")

        
        # Getting Mutual Fund Manager Details
        fund_mgr_Sel="fm982PersonName fs16 fw500 clrText130"
        MF_MGR=html.find_all("div",class_=fund_mgr_Sel)

        
        # Getting Mutual Fund Address Details
        MF_Add_Details_selector="fs16 clrText"
        MF_Add_Details=html.find_all("div",class_=MF_Add_Details_selector)

        
        #Creating list with All Mutual Details 
        Mutual_fund_Details={
        "Mutual Fund Name":MF_Name[0].text,
        "Mutual NAV Date":MF_NAV_date[0].text,
        "Mutual Fund Size":MF_fund_size[2].text,
        "Min Investment":MF_fund_equity_allocation[0].text,
        "Min 2nd Investment":MF_fund_equity_allocation[1].text,
        "Min SIP":MF_fund_equity_allocation[2].text,
        "MF Rank (In Total Asset)":MF_fund_equity_allocation[3].text,
        "Asset under Mgt":MF_fund_equity_allocation[4].text,
        "Date of Incorporation":MF_fund_equity_allocation[5].text,
        "Fund bench mark":MF_fund_equity_allocation[6].text,
        "Exit load":insight_2[1].text,
        "Expense Ratio":MF_fund_Expense_ratio[0].text.strip("Expense ratio: "),
        "Launch Date":MF_Add_Details[2].text,
        "E-Mail":MF_Add_Details[3].text,
        "Mutual Fund Manager":MF_MGR[0].text,
        "Stamped Duty":insight_2[2].text,
        "Tax Implication":insight_2[3].text,
        "Address":MF_Add_Details[0].text,
        "Phone":MF_Add_Details[1].text,

        }
        
        #converting all Details in data frame by using Pandas
        df=pd.DataFrame([Mutual_fund_Details]).T
        df.to_excel(MF_Name[0].text+".xlsx",index=1)
        MF_fund_sector_selection="fs14"

       

        return "Thanks Your Details is converted in Excel Sheet"

    get_topic_page(topic_url)
    

Enter the URL https://groww.in//mutual-funds/navi-nifty-50-index-fund-direct-growth


'Thanks Your Details is converted in Excel Sheet'

# Final Code for getting Asset Allocation with Percentage

In [91]:
# getting Asset Allocation in Various Sector with Percentage of Allocation 

#importing library
import os
import bs4
from bs4 import BeautifulSoup
import requests
import pandas as pd

#getting URL from User for which user want Asset Allocation with per.
topic_url=input("Enter the URL ")

def get_topic_page(topic_url):
        # Download the page
    response = requests.get(topic_url)
    page_content = response.text
        # Check successful response
    if response.status_code != 200:
        raise Exception('Failed to load page {}'.format(topic_url))
        # Parse using Beautiful soup
    
    html= BeautifulSoup(page_content, 'html.parser')
    MF_Name_selector="mfh239SchemeName fw400 fs24"
    MF_Name=html.find_all("h1",class_=MF_Name_selector)
    MF_Name[0].text


    MF_fund_sector=html.find_all("td",class_=MF_fund_sector_selection)
    MF_holding_details={
            "MF Asset":[],
            "Sector":[],
            "Instrument":[],
            "Asset Allocation":[]
            }


    for i in range(len(MF_fund_sector)):
        if i%4==0:
            MF_holding_details["MF Asset"].append((MF_fund_sector[i].text))
        elif i%4==1:
            MF_holding_details["Sector"].append((MF_fund_sector[i].text))
        elif i%4==2:
            MF_holding_details["Instrument"].append((MF_fund_sector[i].text))
        elif i%4==3:
            MF_holding_details["Asset Allocation"].append((MF_fund_sector[i].text))

    holding_Details=pd.DataFrame(MF_holding_details)
    holding_Details.to_excel((MF_Name[0].text)+" Asset Allocation.xlsx",index=0)
    return "Excel Downloaded"

get_topic_page(topic_url)

Enter the URL https://groww.in//mutual-funds/navi-nifty-50-index-fund-direct-growth


'Excel Downloaded'